In [1]:
import os
from os.path import isfile, join
import cv2
import Equirec2Perspec as E2P
import matplotlib.pyplot as plt
import math
import numpy as np
import time
from scipy.io import arff
from scipy import signal, interpolate
import pdb
import time
from scipy.ndimage import zoom
from scipy.special import logsumexp
import tensorflow as tf

# equ = E2P.Equirectangular('src/image.jpg')    # Load equirectangular image

# FOV unit is degree 
# theta is z-axis angle(right direction is positive, left direction is negative)
# phi is y-axis angle(up direction positive, down direction negative)
# height and width is output image dimension 

# img = equ.GetPerspective(100, 0, 0, 1240, 1440)

dirpath = os.getcwd()

In [2]:
def equirectangularToSpherical(x_eq, y_eq, width_px, height_px):
    theta_rad = (x_eq * 2 * math.pi) / width_px
    phi_rad = (-y_eq * math.pi) / height_px
    #theta_rad = (y_eq * math.pi) / height_px
    #phi_rad = (x_eq * 2 * math.pi) / width_px

    theta = theta_rad * (180 / math.pi) - 180
    phi = phi_rad * (180 / math.pi) + 90
    return theta, phi

equirectangularToSpherical(1920, 960, 3840, 1920)

(-2.842170943040401e-14, 1.4210854715202004e-14)

In [3]:
tf.reset_default_graph()

check_point = os.path.join('deep_gaze', 'DeepGazeII.ckpt')  # DeepGaze II
#check_point = 'ICF.ckpt'  # ICF
new_saver = tf.train.import_meta_graph('{}.meta'.format(check_point))

input_tensor = tf.get_collection('input_tensor')[0]
centerbias_tensor = tf.get_collection('centerbias_tensor')[0]
log_density = tf.get_collection('log_density')[0]
log_density_wo_centerbias = tf.get_collection('log_density_wo_centerbias')[0]

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.


In [4]:
dir_train = join(dirpath, "preprocessed_test")

files = sorted([f for f in os.listdir(dir_train) if isfile(join(dir_train, f))])
for i in range(len(files)):
    print(i, files[i])
    
#files = ['002_08_madagascar_cuts.npy']

0 042_01_art_gallery_static.npy
1 042_02_theatre_scene_static.npy
2 042_03_lions_static_cuts.npy
3 042_04_copenhagen_harbour_static_cuts.npy
4 042_05_florida_yacht_motion.npy
5 042_07_factory_robots_static.npy
6 042_08_madagascar_cuts.npy
7 042_11_lohdi_garden_india_cuts.npy
8 042_12_gym_workout_static.npy
9 042_13_times_square_static.npy
10 042_16_interview_static.npy
11 042_17_weather_forecast_static.npy
12 042_18_bomb_trapped_static.npy
13 042_20_car_fix_static.npy
14 042_21_operation_room_static.npy
15 042_23_dog_food_factory_motion.npy
16 043_01_art_gallery_static.npy
17 043_02_theatre_scene_static.npy
18 043_03_lions_static_cuts.npy
19 043_04_copenhagen_harbour_static_cuts.npy
20 043_07_factory_robots_static.npy
21 043_08_madagascar_cuts.npy
22 043_09_elephants_static.npy
23 043_11_lohdi_garden_india_cuts.npy
24 043_13_times_square_static.npy
25 043_16_interview_static.npy
26 043_17_weather_forecast_static.npy
27 043_18_bomb_trapped_static.npy
28 043_20_car_fix_static.npy
29 043_

In [5]:
fove_fov = 100 # 100 degrees
fove_width_px = 720 #1440
fove_height_px = 620 #1240

# load precomputed log density over a 1024x1024 image
centerbias_template = np.load(os.path.join('deep_gaze', 'centerbias.npy'))
# rescale to match image size
centerbias = zoom(centerbias_template, (fove_height_px/1024, fove_width_px/1024), order=0, mode='nearest')
# renormalize log density
centerbias -= logsumexp(centerbias)

centerbias_data = centerbias[np.newaxis, :, :, np.newaxis]  # BHWC, 1 channel (log density)

In [6]:
#columns = ['x', 'y', 'x_head', 'y_head', 'angle_deg_head', 'quaternion_hp_w',
#        'quaternion_hp_x', 'quaternion_hp_y', 'quaternion_hp_z', 'frameId']
with tf.Session() as sess:
    new_saver.restore(sess, check_point)
    start_index = 0
    for file in files[start_index:]:
        start_time_video = time.time()
        video_data = np.load(join(dirpath, "preprocessed_test", file))
        
        video_name = file[4:-4] + ".mp4"
        video_path = join(dirpath, 'datasetTUM', 'videos', video_name)
        print(video_path)
        video_width = 3840
        video_height = video_data[0, 1] * 2
        #x_head_eq = video_data[:, 2]
        #y_head_eq = video_data[:, 3]
        #phi_rad, theta_rad = equirectangularToSpherical(x_head_eq, y_head_eq, video_width, video_height)
        vidcap = cv2.VideoCapture(video_path)

        success, image_frame = vidcap.read()
        frame_id = 0

        img_batch = None
        #img_batch = np.empty((1, fove_height_px, fove_width_px, 3))
        frame_ids = []
        dir_path_fov = join(dirpath, "fov_images_train_hdf5", "original", file[:-4])
        if not os.path.exists(dir_path_fov):
            os.makedirs(dir_path_fov)
        dir_path_fov_saliency = join(dirpath, "fov_images_train_hdf5", "saliency", file[:-4])
        if not os.path.exists(dir_path_fov_saliency):
            os.makedirs(dir_path_fov_saliency)
        start_time = time.time()
        for row in video_data:
            x_head_eq = row[2]
            y_head_eq = row[3]
            theta, phi = equirectangularToSpherical(x_head_eq, y_head_eq, video_width, video_height)
            frame_id_row = row[9]
            if frame_id_row > frame_id:
                success, image_frame = vidcap.read()
                frame_ids.append(frame_id)
                frame_id += 1
                #print(success, frame_id_row, frame_id)
                if success:
                    equ = E2P.Equirectangular(image_frame)
                    # print(row[0], row[1], x_head_eq, y_head_eq, theta, phi)
                    img_fov = equ.GetPerspective(fove_fov, theta, phi, fove_height_px, fove_width_px)
                    img_fov = img_fov[np.newaxis,:,:,:]
                    #downsampled_image = cv2.resize(img,(fove_width_px,fove_height_px))

                    if img_batch is not None:
                        img_batch = np.vstack((img_batch, img_fov))
                    else:
                        img_batch = img_fov
                    #img_batch = np.concatenate((img_batch, img_fov), axis=0)
                    start_time2 = time.time()
                    if len(img_batch) == 15:
                        print(frame_ids)
                        log_density_prediction_batch = sess.run(log_density, {
                            input_tensor: img_batch,
                            centerbias_tensor: centerbias_data,
                        })
                        print("Saliency batch size 15", time.time() - start_time2, "seconds")
                        for i, log_density_prediction in enumerate(log_density_prediction_batch):
                            cv2.imwrite(join(dir_path_fov, "frame%d.jpg" % frame_ids[i]), img_batch[i,:,:,:])
                            max_value = np.amax(np.exp(log_density_prediction))
                            saliencyMap = (np.exp(log_density_prediction) * (255/max_value)).astype("uint8")
                            #m = plt.gca().matshow(np.exp(log_density_prediction[:, :, 0]), cmap=plt.cm.RdBu)
                            #plt.colorbar(m)
                            #print(np.amax(np.exp(log_density_prediction)))
                            cv2.imwrite(join(dir_path_fov_saliency, "frame%d.jpg" % frame_ids[i]), saliencyMap)
                            #np.save(join(dir_path_fov_saliency, "frame%d" % frame_ids[i]), np.exp(log_density_prediction[:,:,0]))
                        img_batch = None
                        #img_batch = np.empty((1, fove_height_px, fove_width_px, 3))
                        frame_ids = []
                        print("Time per batch (15): " + str(time.time() - start_time) + " seconds")
                        start_time = time.time()
        print("Time per video: " + str(time.time() - start_time_video) + " seconds,",
              str((time.time() - start_time_video)/60) + "minutes." )

INFO:tensorflow:Restoring parameters from deep_gaze/DeepGazeII.ckpt
/media/demo/DATA/saliency-exploitation/sw/datasetTUM/videos/01_art_gallery_static.mp4
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
Saliency batch size 15 3.8161838054656982 seconds
Time per batch (15): 5.03207540512085 seconds
[15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
Saliency batch size 15 0.9770543575286865 seconds
Time per batch (15): 2.178072452545166 seconds
[30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]
Saliency batch size 15 0.9730496406555176 seconds
Time per batch (15): 2.176116943359375 seconds
[45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
Saliency batch size 15 0.9771819114685059 seconds
Time per batch (15): 2.2278385162353516 seconds
[60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74]
Saliency batch size 15 0.9737534523010254 seconds
Time per batch (15): 2.2746849060058594 seconds
[75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89]
Sali

Saliency batch size 15 0.9421007633209229 seconds
Time per batch (15): 2.331474542617798 seconds
[720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734]
Saliency batch size 15 0.9382343292236328 seconds
Time per batch (15): 2.337101459503174 seconds
[735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749]
Saliency batch size 15 0.9580845832824707 seconds
Time per batch (15): 2.3019330501556396 seconds
[750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764]
Saliency batch size 15 0.9595365524291992 seconds
Time per batch (15): 2.2051093578338623 seconds
[765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779]
Saliency batch size 15 0.9876339435577393 seconds
Time per batch (15): 2.3125851154327393 seconds
[780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794]
Saliency batch size 15 0.9437422752380371 seconds
Time per batch (15): 2.244204044342041 seconds
[795, 796, 797, 798, 799, 800, 801,

Saliency batch size 15 0.9480257034301758 seconds
Time per batch (15): 2.2727582454681396 seconds
[1395, 1396, 1397, 1398, 1399, 1400, 1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408, 1409]
Saliency batch size 15 0.9453301429748535 seconds
Time per batch (15): 2.308288097381592 seconds
[1410, 1411, 1412, 1413, 1414, 1415, 1416, 1417, 1418, 1419, 1420, 1421, 1422, 1423, 1424]
Saliency batch size 15 0.9381206035614014 seconds
Time per batch (15): 2.3255836963653564 seconds
[1425, 1426, 1427, 1428, 1429, 1430, 1431, 1432, 1433, 1434, 1435, 1436, 1437, 1438, 1439]
Saliency batch size 15 0.9437527656555176 seconds
Time per batch (15): 2.3538620471954346 seconds
[1440, 1441, 1442, 1443, 1444, 1445, 1446, 1447, 1448, 1449, 1450, 1451, 1452, 1453, 1454]
Saliency batch size 15 0.9401788711547852 seconds
Time per batch (15): 2.3402023315429688 seconds
[1455, 1456, 1457, 1458, 1459, 1460, 1461, 1462, 1463, 1464, 1465, 1466, 1467, 1468, 1469]
Saliency batch size 15 0.9414045810699463 seconds
Time p

[225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239]
Saliency batch size 15 0.9463918209075928 seconds
Time per batch (15): 2.216428518295288 seconds
[240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254]
Saliency batch size 15 0.9353501796722412 seconds
Time per batch (15): 2.2865729331970215 seconds
[255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269]
Saliency batch size 15 0.9402000904083252 seconds
Time per batch (15): 2.249962568283081 seconds
[270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284]
Saliency batch size 15 0.9427015781402588 seconds
Time per batch (15): 2.2389638423919678 seconds
[285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299]
Saliency batch size 15 0.9444077014923096 seconds
Time per batch (15): 2.2373433113098145 seconds
[300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314]
Saliency batch size 15 0.9420320987701416 seconds
Time p

Saliency batch size 15 0.9439504146575928 seconds
Time per batch (15): 2.294185161590576 seconds
[945, 946, 947, 948, 949, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959]
Saliency batch size 15 0.9428632259368896 seconds
Time per batch (15): 2.229499578475952 seconds
[960, 961, 962, 963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974]
Saliency batch size 15 0.9409728050231934 seconds
Time per batch (15): 2.2421414852142334 seconds
[975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988, 989]
Saliency batch size 15 0.9471802711486816 seconds
Time per batch (15): 2.2042527198791504 seconds
[990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004]
Saliency batch size 15 0.9454448223114014 seconds
Time per batch (15): 2.2197265625 seconds
[1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019]
Saliency batch size 15 0.9373056888580322 seconds
Time per batch (15): 2.2552692890167236 seconds
[1020, 1021, 1022, 1

...

[825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839]
Saliency batch size 15 0.9447758197784424 seconds
Time per batch (15): 2.230659246444702 seconds
[840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854]
Saliency batch size 15 0.9440333843231201 seconds
Time per batch (15): 2.241454601287842 seconds
[855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869]
Saliency batch size 15 0.9436466693878174 seconds
Time per batch (15): 2.2069735527038574 seconds
[870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884]
Saliency batch size 15 0.9500119686126709 seconds
Time per batch (15): 2.2170145511627197 seconds
[885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899]
Saliency batch size 15 0.9496076107025146 seconds
Time per batch (15): 2.2247650623321533 seconds
[900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914]
Saliency batch size 15 0.9424698352813721 seconds
Time p

Saliency batch size 15 0.9422094821929932 seconds
Time per batch (15): 2.1818928718566895 seconds
[1500, 1501, 1502, 1503, 1504, 1505, 1506, 1507, 1508, 1509, 1510, 1511, 1512, 1513, 1514]
Saliency batch size 15 0.9433605670928955 seconds
Time per batch (15): 2.2351036071777344 seconds
[1515, 1516, 1517, 1518, 1519, 1520, 1521, 1522, 1523, 1524, 1525, 1526, 1527, 1528, 1529]
Saliency batch size 15 0.942927360534668 seconds
Time per batch (15): 2.220019578933716 seconds
[1530, 1531, 1532, 1533, 1534, 1535, 1536, 1537, 1538, 1539, 1540, 1541, 1542, 1543, 1544]
Saliency batch size 15 0.9410216808319092 seconds
Time per batch (15): 2.2376959323883057 seconds
[1545, 1546, 1547, 1548, 1549, 1550, 1551, 1552, 1553, 1554, 1555, 1556, 1557, 1558, 1559]
Saliency batch size 15 0.941953182220459 seconds
Time per batch (15): 2.3036723136901855 seconds
[1560, 1561, 1562, 1563, 1564, 1565, 1566, 1567, 1568, 1569, 1570, 1571, 1572, 1573, 1574]
Saliency batch size 15 0.9454288482666016 seconds
Time per

Saliency batch size 15 0.946685791015625 seconds
Time per batch (15): 2.273925304412842 seconds
[300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314]
Saliency batch size 15 0.9456920623779297 seconds
Time per batch (15): 2.257377862930298 seconds
[315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329]
Saliency batch size 15 0.9430849552154541 seconds
Time per batch (15): 2.1977317333221436 seconds
[330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344]
Saliency batch size 15 0.9361734390258789 seconds
Time per batch (15): 2.246058702468872 seconds
[345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359]
Saliency batch size 15 0.9421801567077637 seconds
Time per batch (15): 2.340505838394165 seconds
[360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374]
Saliency batch size 15 0.9509220123291016 seconds
Time per batch (15): 2.269700527191162 seconds
[375, 376, 377, 378, 379, 380, 381, 38

[1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019]
Saliency batch size 15 0.9430603981018066 seconds
Time per batch (15): 2.3485937118530273 seconds
[1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034]
Saliency batch size 15 0.9429779052734375 seconds
Time per batch (15): 2.2560291290283203 seconds
[1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049]
Saliency batch size 15 0.9370951652526855 seconds
Time per batch (15): 2.331493854522705 seconds
[1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064]
Saliency batch size 15 0.9412980079650879 seconds
Time per batch (15): 2.3658666610717773 seconds
[1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079]
Saliency batch size 15 0.9435646533966064 seconds
Time per batch (15): 2.31949782371521 seconds
[1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 108

Saliency batch size 15 0.939171552658081 seconds
Time per batch (15): 2.307142734527588 seconds
[1665, 1666, 1667, 1668, 1669, 1670, 1671, 1672, 1673, 1674, 1675, 1676, 1677, 1678, 1679]
Saliency batch size 15 0.9436891078948975 seconds
Time per batch (15): 2.2915985584259033 seconds
[1680, 1681, 1682, 1683, 1684, 1685, 1686, 1687, 1688, 1689, 1690, 1691, 1692, 1693, 1694]
Saliency batch size 15 0.9406275749206543 seconds
Time per batch (15): 2.324408769607544 seconds
[1695, 1696, 1697, 1698, 1699, 1700, 1701, 1702, 1703, 1704, 1705, 1706, 1707, 1708, 1709]
Saliency batch size 15 0.9472208023071289 seconds
Time per batch (15): 2.2592413425445557 seconds
[1710, 1711, 1712, 1713, 1714, 1715, 1716, 1717, 1718, 1719, 1720, 1721, 1722, 1723, 1724]
Saliency batch size 15 0.9504539966583252 seconds
Time per batch (15): 2.2713658809661865 seconds
[1725, 1726, 1727, 1728, 1729, 1730, 1731, 1732, 1733, 1734, 1735, 1736, 1737, 1738, 1739]
Saliency batch size 15 0.9368481636047363 seconds
Time per

In [7]:
# plt.gca().imshow(cv2.cvtColor(image_frame, cv2.COLOR_BGR2RGB, alpha=0.2))
#print(log_density_prediction.shape)
#m = plt.gca().matshow(np.exp(log_density_prediction[0, :, :, 0]), cmap=plt.cm.RdBu)
#plt.colorbar(m)
#plt.title('density prediction')
#plt.axis('off');

#print("Time:", time.time() - start_time, "s,", (time.time() - start_time) / 60, "min")